# 目的
- O以外のタグについて、B,Iで結合した文字列を取得し、データフレームにする

In [9]:
import polars as pl
import ast
from tqdm.auto import tqdm

In [10]:
train = pl.read_json("../data/train.json")
moredata = pl.read_csv("../data/moredata_dataset_fixed.csv").with_columns(
    pl.col("tokens").map_elements(lambda x: ast.literal_eval(x)),
    pl.col("trailing_whitespace").map_elements(lambda x: ast.literal_eval(x)),
    pl.col("labels").map_elements(lambda x: ast.literal_eval(x)),
)
pii_dataset = pl.read_csv("../data/pii_dataset_fixed.csv").with_columns(
    pl.col("tokens").map_elements(lambda x: ast.literal_eval(x)),
    pl.col("trailing_whitespace").map_elements(lambda x: ast.literal_eval(x)),
    pl.col("labels").map_elements(lambda x: ast.literal_eval(x)),
)
mixtral = pl.read_json("../data/mixtral-8x7b-v1.json")

In [11]:
# B-で始まっていたら、tokenを文字列に追加、suffixを
# 次のtokenがI-で始まっている & 一つ前と同じ

In [12]:
make_token_df(train[:10])

100%|██████████| 10/10 [00:00<00:00, 3376.24it/s]


NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
str,null,null,null,null,null,null
"""Nathalie Sylla…",null,null,null,null,null,null
"""Nathalie Sylla…",null,null,null,null,null,null
"""Nathalie Sylla…",null,null,null,null,null,null
"""Diego Estrada""",null,null,null,null,null,null
"""Diego Estrada""",null,null,null,null,null,null
"""Gilberto Gambo…",null,null,null,null,null,null
"""Sindy Samaca""",null,null,null,null,null,null
"""Nadine Born""",null,null,null,null,null,null
"""Eladio Amaya""",null,null,null,null,null,null


In [13]:
def tokens_to_entities(tokens, labels):
    entities = []
    entity = []
    current_label = None
    for token, label in zip(tokens, labels):
        if label.startswith("B-"):
            # If there is an entity being built, add it to the entities list
            if entity:
                entities.append((" ".join(entity), current_label))
            # Start a new entity
            entity = [token]
            current_label = label[2:]  # Remove 'B-' from the label
        elif label.startswith("I-") and entity:
            # If the entity is being built, add the token to the entity
            entity.append(token)
    # Add the last entity
    if entity:
        entities.append((" ".join(entity), current_label))
    return entities


def make_token_df(data: pl.DataFrame) -> pl.DataFrame:
    NAME_STUDENT_token = []
    EMAIL_token = []
    USERNAME_token = []
    ID_NUM_token = []
    PHONE_NUM_token = []
    URL_PERSONAL_token = []
    STREET_ADDRESS_token = []

    all_labels = [
        "NAME_STUDENT",
        "EMAIL",
        "USERNAME",
        "ID_NUM",
        "PHONE_NUM",
        "URL_PERSONAL",
        "STREET_ADDRESS",
    ]

    for tokens, labels in tqdm(zip(data["tokens"], data["labels"]), total=len(data)):
        tokens_labels = tokens_to_entities(tokens, labels)
        for token, label in tokens_labels:
            if label == "NAME_STUDENT":
                NAME_STUDENT_token.append(token)
            elif label == "EMAIL":
                EMAIL_token.append(token)
            elif label == "USERNAME":
                USERNAME_token.append(token)
            elif label == "ID_NUM":
                ID_NUM_token.append(token)
            elif label == "PHONE_NUM":
                PHONE_NUM_token.append(token)
            elif label == "URL_PERSONAL":
                URL_PERSONAL_token.append(token)
            elif label == "STREET_ADDRESS":
                STREET_ADDRESS_token.append(token)

    max_len = max(
        len(NAME_STUDENT_token),
        len(EMAIL_token),
        len(USERNAME_token),
        len(ID_NUM_token),
        len(PHONE_NUM_token),
        len(URL_PERSONAL_token),
        len(STREET_ADDRESS_token),
    )

    def pad_token(token: list[str]) -> list[str]:
        return token + [None for _ in range(max_len - len(token))]

    return pl.DataFrame(
        [
            pad_token(NAME_STUDENT_token),
            pad_token(EMAIL_token),
            pad_token(USERNAME_token),
            pad_token(ID_NUM_token),
            pad_token(PHONE_NUM_token),
            pad_token(URL_PERSONAL_token),
            pad_token(STREET_ADDRESS_token),
        ],
        orient="col",
        schema=all_labels,
    )


make_token_df(train)

100%|██████████| 6807/6807 [00:01<00:00, 5532.74it/s]


NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
str,str,str,str,str,str,str
"""Nathalie Sylla…","""djones@gmail.c…","""castanedagabri…","""860632713425""","""( 320)202 - 06…","""https://www.ja…","""591 Smith Cent…"
"""Nathalie Sylla…","""matthew72@hotm…","""fdixon""","""530670102508""","""( 223)392 - 27…","""https://www.li…","""743 Erika Bypa…"
"""Nathalie Sylla…","""belindarojas@y…","""fdixon""","""530670102508""","""( 820)913 - 32…","""https://youtu.…",null
"""Diego Estrada""","""kennethevans@h…","""meyermichelle""","""875673967537""","""( 820)913 - 32…","""https://www.ha…",null
"""Diego Estrada""","""agood@gmail.co…","""jacob59""","""860632713425""","""( 820)913 - 32…","""http://www.bur…",null
"""Gilberto Gambo…","""agood@gmail.co…","""holmespatrick""","""557349702179""","""410.526.1667""","""http://jacobs-…",null
"""Sindy Samaca""","""hwillis@gmail.…",null,"""784372734211""",null,"""https://www.yo…",null
"""Nadine Born""","""kellyharrison@…",null,"""054176622314""",null,"""tps://www.face…",null
"""Eladio Amaya""","""kellyharrison@…",null,"""674915248960""",null,"""https://www.yo…",null


In [14]:
# from tqdm.auto import tqdm


# def make_token_df(data: pl.DataFrame) -> pl.DataFrame:
#     B_NAME_STUDENT_token = []
#     I_NAME_STUDENT_token = []
#     B_EMAIL_token = []
#     I_EMAIL_token = []
#     B_USERNAME_token = []
#     I_USERNAME_token = []
#     B_ID_NUM_token = []
#     I_ID_NUM_token = []
#     B_PHONE_NUM_token = []
#     I_PHONE_NUM_token = []
#     B_URL_PERSONAL_token = []
#     I_URL_PERSONAL_token = []
#     B_STREET_ADDRESS_token = []
#     I_STREET_ADDRESS_token = []

#     all_labels = [
#         "B-NAME_STUDENT",
#         "I-NAME_STUDENT",
#         "B-EMAIL",
#         "I-EMAIL",
#         "B-USERNAME",
#         "I-USERNAME",
#         "B-ID_NUM",
#         "I-ID_NUM",
#         "B-PHONE_NUM",
#         "I-PHONE_NUM",
#         "B-URL_PERSONAL",
#         "I-URL_PERSONAL",
#         "B-STREET_ADDRESS",
#         "I-STREET_ADDRESS",
#     ]

#     for tokens, labels in tqdm(zip(data["tokens"], data["labels"]), total=len(data)):
#         for token, label in zip(tokens, labels):
#             if label == "B-NAME_STUDENT":
#                 B_NAME_STUDENT_token.append(token)
#             elif label == "I-NAME_STUDENT":
#                 I_NAME_STUDENT_token.append(token)
#             elif label == "B-EMAIL":
#                 B_EMAIL_token.append(token)
#             elif label == "I-EMAIL":
#                 I_EMAIL_token.append(token)
#             elif label == "B-USERNAME":
#                 B_USERNAME_token.append(token)
#             elif label == "I-USERNAME":
#                 I_USERNAME_token.append(token)
#             elif label == "B-ID_NUM":
#                 B_ID_NUM_token.append(token)
#             elif label == "I-ID_NUM":
#                 I_ID_NUM_token.append(token)
#             elif label == "B-PHONE_NUM":
#                 B_PHONE_NUM_token.append(token)
#             elif label == "I-PHONE_NUM":
#                 I_PHONE_NUM_token.append(token)
#             elif label == "B-URL_PERSONAL":
#                 B_URL_PERSONAL_token.append(token)
#             elif label == "I-URL_PERSONAL":
#                 I_URL_PERSONAL_token.append(token)
#             elif label == "B-STREET_ADDRESS":
#                 B_STREET_ADDRESS_token.append(token)
#             elif label == "I-STREET_ADDRESS":
#                 I_STREET_ADDRESS_token.append(token)

#     max_len = max(
#         len(B_NAME_STUDENT_token),
#         len(I_NAME_STUDENT_token),
#         len(B_EMAIL_token),
#         len(I_EMAIL_token),
#         len(B_USERNAME_token),
#         len(I_USERNAME_token),
#         len(B_ID_NUM_token),
#         len(I_ID_NUM_token),
#         len(B_PHONE_NUM_token),
#         len(I_PHONE_NUM_token),
#         len(B_URL_PERSONAL_token),
#         len(I_URL_PERSONAL_token),
#         len(B_STREET_ADDRESS_token),
#         len(I_STREET_ADDRESS_token),
#     )

#     def pad_token(token: list[str]) -> list[str]:
#         return token + [None for _ in range(max_len - len(token))]

#     return pl.DataFrame(
#         [
#             pad_token(B_NAME_STUDENT_token),
#             pad_token(I_NAME_STUDENT_token),
#             pad_token(B_EMAIL_token),
#             pad_token(I_EMAIL_token),
#             pad_token(B_USERNAME_token),
#             pad_token(I_USERNAME_token),
#             pad_token(B_ID_NUM_token),
#             pad_token(I_ID_NUM_token),
#             pad_token(B_PHONE_NUM_token),
#             pad_token(I_PHONE_NUM_token),
#             pad_token(B_URL_PERSONAL_token),
#             pad_token(I_URL_PERSONAL_token),
#             pad_token(B_STREET_ADDRESS_token),
#             pad_token(I_STREET_ADDRESS_token),
#         ],
#         orient="col",
#         schema=all_labels,
#     )

In [15]:
train_token_df = make_token_df(train)

100%|██████████| 6807/6807 [00:01<00:00, 5614.92it/s]


In [16]:
moredata_token_df = make_token_df(moredata)
moredata_token_df.columns = [
    "more_" + col_name for col_name in moredata_token_df.columns
]

100%|██████████| 2000/2000 [00:00<00:00, 7174.79it/s]


In [17]:
pii_dataset_token_df = make_token_df(pii_dataset)
pii_dataset_token_df.columns = [
    "pii_" + col_name for col_name in pii_dataset_token_df.columns
]

100%|██████████| 4434/4434 [00:00<00:00, 10710.15it/s]


In [18]:
mixtral_token_df = make_token_df(mixtral)
mixtral_token_df.columns = [
    "mixtral_" + col_name for col_name in mixtral_token_df.columns
]

100%|██████████| 2355/2355 [00:00<00:00, 4587.99it/s]


In [19]:
token_df = pl.concat(
    [train_token_df, moredata_token_df, pii_dataset_token_df, mixtral_token_df],
    how="horizontal",
)

In [20]:
token_df_column = []
for train_token, modedata_token, pii_dataset_token, mixtral_token in zip(
    train_token_df.columns,
    moredata_token_df.columns,
    pii_dataset_token_df.columns,
    mixtral_token_df.columns,
):
    token_df_column.append(train_token)
    token_df_column.append(modedata_token)
    token_df_column.append(pii_dataset_token)
    token_df_column.append(mixtral_token)

In [21]:
# 並び替え
token_df[token_df_column].write_csv("output/concat_token_df_with_mixtral.csv")

In [22]:
train_token_df

NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
str,str,str,str,str,str,str
"""Nathalie Sylla…","""djones@gmail.c…","""castanedagabri…","""860632713425""","""( 320)202 - 06…","""https://www.ja…","""591 Smith Cent…"
"""Nathalie Sylla…","""matthew72@hotm…","""fdixon""","""530670102508""","""( 223)392 - 27…","""https://www.li…","""743 Erika Bypa…"
"""Nathalie Sylla…","""belindarojas@y…","""fdixon""","""530670102508""","""( 820)913 - 32…","""https://youtu.…",null
"""Diego Estrada""","""kennethevans@h…","""meyermichelle""","""875673967537""","""( 820)913 - 32…","""https://www.ha…",null
"""Diego Estrada""","""agood@gmail.co…","""jacob59""","""860632713425""","""( 820)913 - 32…","""http://www.bur…",null
"""Gilberto Gambo…","""agood@gmail.co…","""holmespatrick""","""557349702179""","""410.526.1667""","""http://jacobs-…",null
"""Sindy Samaca""","""hwillis@gmail.…",null,"""784372734211""",null,"""https://www.yo…",null
"""Nadine Born""","""kellyharrison@…",null,"""054176622314""",null,"""tps://www.face…",null
"""Eladio Amaya""","""kellyharrison@…",null,"""674915248960""",null,"""https://www.yo…",null
